# 第4章 word2vecの高速化

## 4.1 word2vecの改良①

### 4.1.1 Embeddingレイヤ
シンプルなCBOWモデルでは実装に問題がある．
例えば，単語数が100万語あるとき，$W_{in}$，$W_{out}$，softmaxの計算ばボトルネックになる．
そこで，Embeddingレイヤを導入する．

### 4.1.2 Embeddingレイヤの実装
matmalレイヤでは，列ベクトルと行列の演算を行っているが，意味としては，行列の単に行列の特定の行を抽出しているだけである．

単語IDに該当する行を抜き出すためのレイヤーを作成する．
* embeddingは単語の埋め込みという用語に由来する．
* Embeddingレイヤに単語の埋め込み，分散表現が格納される．

In [ ]:
import numpy as np
import sys
sys.path.append('../deep-learning-from-scratch-2')
from common.config import GPU

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        if GPU:
            import cupy as cp
            cp.scatter_add(dW, self.idx, dout)
        else:
            np.add.at(dW, self.idx, dout)
        return None

## 4.2 word2vecの改良②
中間層以降の処理
ネガティブサンプリングを行う

### 4.2.1 中間層以降の計算の問題点
中間層と重みの積
softmaxレイヤの計算

### 4.2.2 多値分類から二値分類へ
多値分類を２値分類で近似するアイデア

これまでやってきたことは，コンテキストから正解となる単語を高い確率で推測できるようにすること．

これは多値分類だが，これを２値分類に変更する．
* コンテキストがyouとgoodbyeとするときにターゲットとなる単語はsayですかという質問に変える．

出力層のニューロンは一つだけで，中間層と出力側の重みの行列の積において，sayに対応する列だけを抽出すれば良い．

### 4.2.3 シグモイド関数と交差エントロピー誤差
多値分類においては，softmax関数と交差エントロピー誤差を用いる．２知分類では，sigmoid関数と交差エントロピー誤差を用いる．sigmoid関数は以下の式である．
$$\sigma(x) = \frac{1}{1 + e^{-x}}
$$
交差エントロピー誤差は以下の式である．
$$L = -(t \log y + (1 -t) \log (1-y)) $$
tは正解ラベル．tが1の時は，正解がyes，tが0の時は，正解がno．
tが1の時は，-logyが出力される．tが0の時は，-log(1-y)が出力される．
逆伝播において，

### 4.2.4 多値分類から二値分類へ（実装編）
$W_{out}$においてもembeddingの機能を持たせる．

embedはembeddingレイヤをparamにはパラメータをgradsには勾配を格納する．cacheは順伝播の際に必要な情報を保持する．


In [2]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 4.2.5 Negative Sampling
負の例の方は対象となっていない．
そのため，ネガティブな例を少数サンプリングしてLossを計算する




### 4.2.6 Negative Samplingのサンプリング手法

コーパスの統計データを用いて，使われやすい単語は抽出されやすくし，あまり使われないデータは抽出されないようにする．
各ちる分布に対して，一手間加えた値を用いて，サンプリングを行う．
$$
P^{'}(w_i) = \frac{P( w_i)^{3/4}}{\sum_j P(w_j)^{3/4}}
$$

出現確率が低い単語を見捨てないようにするため


UnigramSampler
クラスで実装する．
ユニグラムは，一つの単語を意味する．
バイグラムは，２つの連続した単語を意味する．例えば，(you, say)や(you goodbye)などの２つの単語の組み合わせを対象とした確率分布を作るということ．

### 4.2.7 Negative Samplingの実装

In [ ]:
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）で計算するときは、速度を優先
            # 負例にターゲットが含まれるケースがある
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample


class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 正例のフォワード
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 負例のフォワード
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

## 4.3 改良版word2vecの学習

### 4.3.1 CBOWモデルの実装


In [4]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embeddingレイヤを使用
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # すべての重みと勾配をリストにまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

## 4.4 word2vecに関する残りのテーマ

### 4.4.1 word2vecを使ったアプリケーションの例

類似単語を求める用途に利用できる．
しかし，単語の分散表現が重要な理由は転移学習(transfer learning)にある．自然言語のタスクを解く際に，単語の分散表現をゼロから学習することはほとんどない．そうではなく，先に大きなコーパスで学習を行い，その学習済みの分散表現を個別のタスクで利用する．例えば，テキスト分類や文書クラスタりんぐ，品詞タグ付け，感情分析などの自然言語のタスクにおいて，単語をベクトルに変換する最初のステップでは，学習済みの単語の分散表現を利用することができる．その多種多様な自然言語処理のタスクのほとんど全てにおいて，単語の分散表現は素晴らしい結果をもたらす．

また，単語の分散表現の利点は，単語を固定長のベクトルに変換できることにある．さらに，文章（単語の並び）に対しても，単語の分散表現を使って固定長のベクトルに変換できる．RNNを用いることによって文章を固定長のベクトルに変換できる

単語を固定長のベクトルに変換できることによって，一般的な機械学習の手法が適用できるから非常に重要である．


### 4.4.2 単語ベクトルの評価方法

単語の分散表現を得ることができるが，これの良さはどのように評価するべきか．

現実的には，何かしらのアプリケーションにおいて，単語の分散表現が使われる際に，最終的に望まれるのは精度の良いシステムである．ここで考えなければいけないことは，システムが複数のシステムから構成されるということ．例えば，単語の分散表現を作るシステムと特定の問題に対して分類を行うシステムがある．この場合は，２段階の学習を行なった上で，評価する必要がある．しかし，これには２つのシステムにおいてハイパーパラメータのためのチューニングが必要であり，多くの時間がかかってしまう．

そこで，単語の分散表現の評価を行うにあたり，現実的なアプリケーションとは切り離して評価を行うというのが一般的によく行われる．その際によく用いられる評価指標が類似性や類推問題である．

単語類似性の評価では，人間が作成した単語類似度の評価セットを使って評価することが多く行われる．例えば，catとanimalの類似度は0.8，catとcarの類似度は0.2などのように，人間が評価した単語の類似度を用いて，単語ベクトルの類似度を評価する．

類推問題による評価は，king:queen = man: ?のような類推問題を出題し，その正解率でもって単語の分散表現の良さを図る．類推問題によって，単語の意味や文法的な問題を正しく理解しているかということをある程度計測することができる．そのため，類推問題を精度良く解くことのできる単語の分散表現であれば，自然言語を扱うアプリケーションにおいても良い結果が期待できる．

## 4.5 まとめ
* Embeddingレイヤは単語の分散表現を格納し，順伝播において，該当する単語のIDのベクトルを抽出する．
* word2vecでは，語彙数の増加に比例して計算量が増加するので，近似計算を行う高速な手法を使うと良い
* Negative Samplingは負の例をいくつかサンプリングする手法であり，これを利用すれば多値分類を2値分類として扱うことができる．
* word2vecで得られた単語の分散表現は，単語の意味が埋め込まれたものであり，似たコンテキストで使われる単語は単語のベクトルの空間として近い場所に位置するようになる
* word2vecの単語の分散表現は類推問題をベクトルの加算と減算によって解ける性質を持つ
* word2vecの単語の分散表現は類推問題をベクトルの加算と減算によって解ける性質を持つ
* word2vecは転移学習の点で特に重要であり，その単語の分散表現はさまざまな自然言語処理のタスクに利用できる．

